In [ ]:
pip install openai

Note: you may need to restart the kernel to use updated packages.


In [ ]:
#  0. 라이브러리 임포트 및 API 키 입력
from openai import OpenAI
import os

#  사용자에게 키 입력받기
api_key = input(" OpenAI Key를 입력하세요: ").strip()

#  클라이언트 초기화
client = OpenAI(api_key=api_key)


In [ ]:
# 1. train.jsonl 파일 업로드
train_file = client.files.create(
    file=open("train.jsonl", "rb"),
    purpose="fine-tune"
)
print("OK! Train File ID:", train_file.id)


✅ Train File ID: file-35AHAiPBwZihd5bHHY2uAX


In [ ]:
# 2. test.jsonl 파일 업로드 (선택)
val_file = client.files.create(
    file=open("test.jsonl", "rb"),
    purpose="fine-tune"
)
print("OK! Validation File ID:", val_file.id)


✅ Validation File ID: file-Kpqwv7AuA2FUCEerGKPr98


In [ ]:
# 3. 파인튜닝 시작
fine_tune_job = client.fine_tuning.jobs.create(
    training_file=train_file.id,
    validation_file=val_file.id,
    model="gpt-3.5-turbo",
    suffix="family-diary"
)

print("OK! 파인튜닝 Job ID:", fine_tune_job.id)


🚀 파인튜닝 Job ID: ftjob-lm7dt11xOttUGjakPEQnELxu


In [ ]:
# 4. 파인튜닝 상태 확인
job_id = fine_tune_job.id
status = client.fine_tuning.jobs.retrieve(job_id)
print("OK! 현재 상태:", status.status)


📊 현재 상태: validating_files


In [32]:
# 가장 최근 파인튜닝 job 정보 확인
jobs = client.fine_tuning.jobs.list(limit=5)

for job in jobs.data:
    print("🔹 모델 이름:", job.fine_tuned_model)


🔹 모델 이름: None
🔹 모델 이름: None
🔹 모델 이름: ft:gpt-3.5-turbo-0125:personal:family-diary:BedVJIlc
🔹 모델 이름: None
🔹 모델 이름: None


In [ ]:
# 5. 파인튜닝 완료 후, 모델로 테스트
completion = client.chat.completions.create(
    model="ft:gpt-3.5-turbo-0125:personal:family-diary:BedVJIlc",  # 실제 완성된 모델명으로...
    messages=[
        {"role":"user","content":"엄마: 애들은 다들 일어났니?\n엄마: 엄마는 할머니 병원 왔다.\n엄마: 태건이는 어디야? \n태건: 가는중\n엄마: 이모티콘\n엄마: 아빠는 왜 답없대?\n태건: 아빠가 태워주는겨\n엄마: 웬일 태워다준대?ㅎ\n채원: 오홍\n채원: 아빠 출근 안한겨~?\n엄마: 아빠 10시에 고대로\n엄마: 바로 출근\n엄마: 째옹 출근준비중?\n채원: 넹~\n채원: 서울울대 가용\n엄마: 오늘도 가는겨? 혼자?\n채원: 애들이랑!\n엄마: ㅇㅋ\n엄마: 바람이 너무 분다. 다들 조심\n채원: 안날라 가게 조심할게요~\n채원: 이모티콘"}
    ]
)

print("OK! 요약 결과:", completion.choices[0].message.content)


📝 요약 결과: 오늘은 엄마가 할머니 병원을 오가며 바쁜 하루를 보냈고, 가족들은 하나둘 일어나며 서로 소통했다. 채원과 태건은 서울 울대에 가는 길, 바람이 세다며 서로 조심 인사를 나눴다. 오늘도 가족 간 따뜻한 소통이 오가는 하루였다.
